In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('DECEMBER.csv')
df.head()

,stafff,shop,remarks,CLIENT_NAME,phone_number,Iphone,phone_id,deposit,installment,contract_date,...,payment_20,payment_20_Status,payment_21,payment_21_Status,payment_22,payment_22_Status,payment_23,payment_23_Status,payment_24,payment_24_Status
0,AMOS,WIRELESS GARAGE,"fully paid, cleared with 1.25 discount",LINDA MAINA,718464661,12 PRO 25G6GB,G6TF67S00D86,24000,4500.0,12/2/2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DERRICK,APPLEGEEKS,NaN,SYLVIA SOILA,742741682,11 PRO 64GB,C39ZNU4GN6Y5,14000,2625.0,12/2/2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AMOS,IPHONEHOUSE,"Locked on 19th Dec, Lost iphone",JEFF NJINE,769962375,14 PMAX 256GB,FXK33JWM74,68000,4625.0,12/2/2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,DERRICK,FANCYTECH,overpay 4400,RUTH NJOKI KAHUNGU,707278156,12 PRO 256GB,F17DL2AL0D9K,27200,5100.0,12/2/2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ALIPS,BITECH,"TO BE PAYING ON WEDNESDAYS,overpay 20",DENIS OTIENO/VICTOR ODUOR,706270800/708659145,13 128GB,MDXTHCX40G,25000,2812.5,12/2/2024,...,4/21/2025,Pending,4/28/2025,Pending,5/5/2025,Pending,5/12/2025,Pending,5/19/2025,Pending


In [3]:
# Columns to consider for payments
payment_columns = [col for col in df.columns if 'payment_' in col and 'Status' not in col]
status_columns = [col for col in df.columns if 'Status' in col]

In [4]:
# Melt the DataFrame to unpivot payment dates and statuses
melted_payments = df.melt(
    id_vars=['installment'],
    value_vars=payment_columns,
    var_name='payment_period',
    value_name='payment_date'
)

melted_statuses = df.melt(
    id_vars=['installment'],
    value_vars=status_columns,
    var_name='payment_status_period',
    value_name='payment_status'
)


In [5]:
# Combine the melted payments and statuses
combined = pd.concat([melted_payments, melted_statuses['payment_status']], axis=1)

In [6]:
# Extract month and year from payment dates
combined['month_year'] = pd.to_datetime(combined['payment_date'], errors='coerce').dt.to_period('M')

In [7]:
# Calculate expected and actual amounts
combined['expected_amount'] = combined['installment']
combined['actual_paid'] = combined.apply(
    lambda row: row['installment'] if row['payment_status'] in ['Paid', 'Returned'] else 0, axis=1
)

In [8]:
# Group by month and year
monthly_summary = combined.groupby('month_year').agg(
    expected_amount=('expected_amount', 'sum'),
    actual_paid=('actual_paid', 'sum')
).reset_index()

In [9]:
# Display the result
print(monthly_summary)

  month_year  expected_amount  actual_paid
0    2024-12        3910237.0    3454375.0
1    2025-01        8326790.0    1973380.0
2    2025-02        7254980.0     431260.0
3    2025-03        3555730.5     111875.0
4    2025-04         645262.0      28260.0
5    2025-05         420127.5      25470.0
6    2025-06           5715.0          0.0


In [10]:
# Calculate % Achievement
monthly_summary['percentage_achievement'] = (
    monthly_summary['actual_paid'] / monthly_summary['expected_amount'] * 100
)

# Fill NaN values (e.g., if expected_amount is 0) with 0%
monthly_summary['percentage_achievement'] = monthly_summary['percentage_achievement'].fillna(0)

# Display the result with % Achievement
print(monthly_summary)


  month_year  expected_amount  actual_paid  percentage_achievement
0    2024-12        3910237.0    3454375.0               88.341832
1    2025-01        8326790.0    1973380.0               23.699169
2    2025-02        7254980.0     431260.0                5.944331
3    2025-03        3555730.5     111875.0                3.146330
4    2025-04         645262.0      28260.0                4.379616
5    2025-05         420127.5      25470.0                6.062445
6    2025-06           5715.0          0.0                0.000000


In [11]:
import pandas as pd

# Example DataFrame
data = {
    'month_year': ['2024-12', '2025-01', '2025-02', '2025-03'],
    'expected_amount': [100000, 120000, 100000, 150000],
    'actual_paid': [90000, 84000, 71000, 108000]
}

df = pd.DataFrame(data)

# Calculate percentage achievement
df['percentage_achievement'] = (df['actual_paid'] / df['expected_amount']) * 100

# Define a function to calculate commission
def calculate_commission(percentage):
    if percentage < 70:
        return 0  # No commission if achievement is below 70%
    else:
        return 10000 + (percentage - 70) * 1000  # Base 10,000 + 1,000 for every point above 70%

# Apply the function to calculate commission
df['commission'] = df['percentage_achievement'].apply(calculate_commission)

# Display the DataFrame
print(df)


  month_year  expected_amount  actual_paid  percentage_achievement  commission
0    2024-12           100000        90000                    90.0     30000.0
1    2025-01           120000        84000                    70.0     10000.0
2    2025-02           100000        71000                    71.0     11000.0
3    2025-03           150000       108000                    72.0     12000.0


In [12]:
df.columns

Index(['month_year', 'expected_amount', 'actual_paid',
       'percentage_achievement', 'commission'],
      dtype='object')

In [13]:
!pip install flask pyngrok


In [14]:
from flask import Flask, request, jsonify, render_template_string
from pyngrok import ngrok

In [ ]:
from flask import Flask, request, render_template_string
import pandas as pd

app = Flask(__name__)

# Define HTML template for the page
HTML_TEMPLATE = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>CSV Summary Generator</title>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/papaparse/5.4.1/papaparse.min.js"></script>
    <style>
        body {
            font-family: Arial, sans-serif;
        }
        .summary-table {
            width: 100%;
            border-collapse: collapse;
            margin: 20px 0;
        }
        .summary-table th, .summary-table td {
            border: 1px solid #ddd;
            padding: 8px;
        }
        .summary-table th {
            background-color: #f2f2f2;
        }
    </style>
</head>
<body>
    <h1>CSV Summary Generator</h1>
    <form id="upload-form" action="/process" method="POST" enctype="multipart/form-data">
        <input type="file" name="csvFileInput" accept=".csv" />
        <button type="submit">Generate Summary</button>
    </form>

    {% if results %}
    <table id="summaryTable" class="summary-table">
        <thead>
            <tr>
                <th>Month</th>
                <th>Total Expected Amount</th>
                <th>Total Actual Amount</th>
                <th>Average Achievement (%)</th>
                <th>Commission</th>
            </tr>
        </thead>
        <tbody>
            {% for result in results %}
            <tr>
                <td>{{ result['Month'] }}</td>
                <td>{{ result['Total_Expected_Amount'] }}</td>
                <td>{{ result['Total_Actual_Amount'] }}</td>
                <td>{{ result['Average_Achievement'] }}</td>
                <td>{{ result['Commission'] }}</td>
            </tr>
            {% endfor %}
        </tbody>
    </table>
    {% endif %}
</body>
</html>
'''

@app.route('/')
def index():
    return render_template_string(HTML_TEMPLATE)

@app.route('/process', methods=['POST'])
def process_csv():
    if 'csvFileInput' not in request.files:
        return "No file part", 400

    file = request.files['csvFileInput']
    if file.filename == '':
        return "No selected file", 400

    # Read CSV into a DataFrame
    df = pd.read_csv(file)

    # Initialize the summary dictionary to aggregate the values by month
    monthly_summary = {}

    # Define a function to calculate the commission
    def calculate_commission(percentage):
        if percentage < 70:
            return 0  # No commission if achievement is below 70%
        else:
            return 10000 + (percentage - 70) * 1000  # Base 10,000 + 1,000 for every point above 70%

    # Iterate through each row and calculate monthly sums
    for _, row in df.iterrows():
        for i in range(1, 25):  # Assuming 24 payments (payment_1 to payment_24)
            payment_column = f"payment_{i}"
            payment_status_column = f"payment_{i}_Status"

            # Check if there's a payment entry for this column
            if pd.notna(row[payment_column]):
                # Extract the month and year from the payment date
                month = pd.to_datetime(row[payment_column]).strftime('%Y-%m')

                # Calculate expected and actual amounts
                expected_amount = row['installment']  # Since installment is the same for each row
                actual_amount = row['installment'] if row[payment_status_column] in ["Paid", "Returned"] else 0

                # If the month doesn't exist in the dictionary, initialize it
                if month not in monthly_summary:
                    monthly_summary[month] = {
                        "Total_Expected_Amount": 0,
                        "Total_Actual_Amount": 0,
                        "count": 0
                    }

                # Aggregate the values
                monthly_summary[month]["Total_Expected_Amount"] += expected_amount
                monthly_summary[month]["Total_Actual_Amount"] += actual_amount
                monthly_summary[month]["count"] += 1

    # Calculate the achievement, commission, and prepare the summary results
    results = []
    for month, data in monthly_summary.items():
        total_expected = data["Total_Expected_Amount"]
        total_actual = data["Total_Actual_Amount"]
        achievement = (total_actual / total_expected) * 100 if total_expected > 0 else 0
        commission = calculate_commission(achievement)
        results.append({
            "Month": month,
            "Total_Expected_Amount": total_expected,
            "Total_Actual_Amount": total_actual,
            "Average_Achievement": f"{achievement:.2f}%",
            "Commission": f"{commission:.2f}"
        })

    return render_template_string(HTML_TEMPLATE, results=results)

if __name__ == "__main__":
    app.run(debug=True, use_reloader=False, port=5000)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [22/Jan/2025 10:07:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Jan/2025 10:07:29] "POST /process HTTP/1.1" 200 -


In [ ]:
# from flask import Flask, request, render_template_string
# import pandas as pd

# app = Flask(__name__)

# # Define HTML template for the page
# HTML_TEMPLATE = '''
# <!DOCTYPE html>
# <html lang="en">
# <head>
#     <meta charset="UTF-8">
#     <meta name="viewport" content="width=device-width, initial-scale=1.0">
#     <title>CSV Summary Generator</title>
#     <script src="https://cdnjs.cloudflare.com/ajax/libs/papaparse/5.4.1/papaparse.min.js"></script>
#     <style>
#         body {
#             font-family: Arial, sans-serif;
#         }
#         .summary-table {
#             width: 100%;
#             border-collapse: collapse;
#             margin: 20px 0;
#         }
#         .summary-table th, .summary-table td {
#             border: 1px solid #ddd;
#             padding: 8px;
#         }
#         .summary-table th {
#             background-color: #f2f2f2;
#         }
#     </style>
# </head>
# <body>
#     <h1>CSV Summary Generator</h1>
#     <form id="upload-form" action="/process" method="POST" enctype="multipart/form-data">
#         <input type="file" name="csvFileInput" accept=".csv" />
#         <button type="submit">Generate Summary</button>
#     </form>

#     {% if results %}
#     <table id="summaryTable" class="summary-table">
#         <thead>
#             <tr>
#                 <th>Month</th>
#                 <th>Total Expected Amount</th>
#                 <th>Total Actual Amount</th>
#                 <th>Average Achievement (%)</th>
#             </tr>
#         </thead>
#         <tbody>
#             {% for result in results %}
#             <tr>
#                 <td>{{ result['Month'] }}</td>
#                 <td>{{ result['Total_Expected_Amount'] }}</td>
#                 <td>{{ result['Total_Actual_Amount'] }}</td>
#                 <td>{{ result['Average_Achievement'] }}</td>
#             </tr>
#             {% endfor %}
#         </tbody>
#     </table>
#     {% endif %}
# </body>
# </html>
# '''

# @app.route('/')
# def index():
#     return render_template_string(HTML_TEMPLATE)

# @app.route('/process', methods=['POST'])
# def process_csv():
#     if 'csvFileInput' not in request.files:
#         return "No file part", 400

#     file = request.files['csvFileInput']
#     if file.filename == '':
#         return "No selected file", 400

#     # Read CSV into a DataFrame
#     df = pd.read_csv(file)

#     # Initialize the summary dictionary to aggregate the values by month
#     monthly_summary = {}

#     # Iterate through each row and calculate monthly sums
#     for _, row in df.iterrows():
#         for i in range(1, 25):  # Assuming 24 payments (payment_1 to payment_24)
#             payment_column = f"payment_{i}"
#             payment_status_column = f"payment_{i}_Status"

#             # Check if there's a payment entry for this column
#             if pd.notna(row[payment_column]):
#                 # Extract the month and year from the payment date
#                 month = pd.to_datetime(row[payment_column]).strftime('%Y-%m')

#                 # Calculate expected and actual amounts
#                 expected_amount = row['installment']  # Since installment is the same for each row
#                 actual_amount = row['installment'] if row[payment_status_column] in ["Paid", "Returned"] else 0

#                 # If the month doesn't exist in the dictionary, initialize it
#                 if month not in monthly_summary:
#                     monthly_summary[month] = {
#                         "Total_Expected_Amount": 0,
#                         "Total_Actual_Amount": 0,
#                         "count": 0
#                     }

#                 # Aggregate the values
#                 monthly_summary[month]["Total_Expected_Amount"] += expected_amount
#                 monthly_summary[month]["Total_Actual_Amount"] += actual_amount
#                 monthly_summary[month]["count"] += 1

#     # Calculate the achievement for each month and prepare the summary results
#     results = []
#     for month, data in monthly_summary.items():
#         total_expected = data["Total_Expected_Amount"]
#         total_actual = data["Total_Actual_Amount"]
#         achievement = (total_actual / total_expected) * 100 if total_expected > 0 else 0
#         results.append({
#             "Month": month,
#             "Total_Expected_Amount": total_expected,
#             "Total_Actual_Amount": total_actual,
#             "Average_Achievement": f"{achievement:.2f}%"
#         })

#     return render_template_string(HTML_TEMPLATE, results=results)

# if __name__ == "__main__":
#     app.run(debug=True, use_reloader=False, port=5000)
